In [1]:
# 경고(warning) 비표시
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

# 제3장 고객의 전체모습을 파악하는 테크닉 10

### 테크닉21 : 데이터를 읽어들이고 확인하자

In [2]:
use_log = pd.read_csv('use_log.csv')
print(len(use_log))
use_log.head()

197428


,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


In [3]:
customer_master = pd.read_csv('customer_master.csv')
print(len(customer_master))
customer_master.head()

4192


,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0


In [4]:
class_master = pd.read_csv('class_master.csv')
print(len(class_master))
class_master.head()

3


,class,class_name,price
0,C01,0_종일,10500
1,C02,1_주간,7500
2,C03,2_야간,6000


In [5]:
campaign_master = pd.read_csv('campaign_master.csv')
print(len(campaign_master))
campaign_master.head()

3


,campaign_id,campaign_name
0,CA1,2_일반
1,CA2,0_입회비반액할인
2,CA3,1_입회비무료


### 테크닉22 : 고객 데이터를 가공하자

In [6]:
user_type = pd.merge(customer_master, class_master, how='left', on='class')
print(len(user_type))
user_type = pd.merge(user_type, campaign_master, how='left', on='campaign_id')
user_type.groupby('class_name').count()['customer_id']

4192


class_name
0_종일    2045
1_주간    1019
2_야간    1128
Name: customer_id, dtype: int64

In [7]:
user_type.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,2_야간,6000,2_일반


In [8]:
user_type.isnull().sum()

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
dtype: int64

### 테크닉23 : 고객 데이터를 집계하자

In [9]:
user_type.groupby('class_name').count()['customer_id']

class_name
0_종일    2045
1_주간    1019
2_야간    1128
Name: customer_id, dtype: int64

In [10]:
user_type.groupby('campaign_name').count()['customer_id']

campaign_name
0_입회비반액할인     650
1_입회비무료       492
2_일반         3050
Name: customer_id, dtype: int64

In [11]:
user_type.groupby(['is_deleted', 'campaign_name']).count()['customer_id']

is_deleted  campaign_name
0           0_입회비반액할인         282
            1_입회비무료           219
            2_일반             2341
1           0_입회비반액할인         368
            1_입회비무료           273
            2_일반              709
Name: customer_id, dtype: int64

In [12]:
user_type['start_date'] = pd.to_datetime(user_type['start_date'])
user_start = user_type.loc[user_type['start_date'] > pd.to_datetime('20180401')]
print(len(user_type))
print(len(user_start))

4192
1361


### 테크닉24 : 최근 고객데이터를 집계해보자

In [13]:
user_type['end_date'] = pd.to_datetime(user_type['end_date'])
user_newer = user_type.loc[(user_type['end_date'] >= pd.to_datetime("20190331")) | (user_type['end_date'].isna())]
user_newer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반
1,PL270116,XXXXX,C01,M,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반
2,OA974876,XXXXX,C01,M,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반
3,HD024127,XXXXX,C01,F,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반
4,HD661448,XXXXX,C03,F,2015-05-01,NaT,CA1,0,2_야간,6000,2_일반
...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaT,CA1,0,0_종일,10500,2_일반
4188,HD246549,XXXXX,C01,F,2019-03-14,NaT,CA1,0,0_종일,10500,2_일반
4189,GD037007,XXXXX,C03,M,2019-03-14,NaT,CA1,0,2_야간,6000,2_일반
4190,OA953150,XXXXX,C01,M,2019-03-14,NaT,CA1,0,0_종일,10500,2_일반


In [14]:
user_newer.groupby('class_name').count()['customer_id']

class_name
0_종일    1444
1_주간     696
2_야간     813
Name: customer_id, dtype: int64

### 테크닉25 : 이용이력 데이터를 집계하자

In [15]:
use_log['usedate'] = pd.to_datetime(use_log['usedate'])
use_log['연월'] = use_log['usedate'].dt.strftime('%Y%m')
use_log.head()

,log_id,customer_id,usedate,연월
0,L00000049012330,AS009373,2018-04-01,201804
1,L00000049012331,AS015315,2018-04-01,201804
2,L00000049012332,AS040841,2018-04-01,201804
3,L00000049012333,AS046594,2018-04-01,201804
4,L00000049012334,AS073285,2018-04-01,201804


In [16]:
uselogMonth = use_log.groupby(['연월','customer_id'], as_index=False).count()
uselogMonth.rename(columns={'log_id':'count'}, inplace=True)

# 특정 컬럼 삭제
del uselogMonth['usedate']
uselogMonth

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36837,201903,TS995853,8
36838,201903,TS998593,8
36839,201903,TS999079,3
36840,201903,TS999231,6


In [17]:
uselogMonth = uselogMonth.groupby('customer_id').agg(['mean', 'median', 'max', 'min'])['count']

In [18]:
user_type = pd.merge(user_type, uselogMonth, how='left', on='customer_id')
print(len(user_type['customer_id'].unique()))

4192


### 테크닉26 : 이용이력 데이터로부터 정기이용 여부  플래그를 작성하자

In [19]:
use_log['weekday'] = use_log['usedate'].dt.weekday

use_log_weekly = use_log.groupby(['customer_id','연월','weekday'], as_index=False).count()[['customer_id','연월','weekday','log_id']]
use_log_weekly.rename(columns={'log_id':'count'}, inplace=True)
use_log_weekly.head()

,customer_id,연월,weekday,count
0,AS002855,201804,5,4
1,AS002855,201805,2,1
2,AS002855,201805,5,4
3,AS002855,201806,5,5
4,AS002855,201807,1,1


In [20]:
use_log_weekly = use_log_weekly.groupby('customer_id', as_index=False).max()[['customer_id','count']]
use_log_weekly['routine_flag'] = 0
use_log_weekly.head(30)

,customer_id,count,routine_flag
0,AS002855,5,0
1,AS008805,4,0
2,AS009013,2,0
3,AS009373,5,0
4,AS015233,5,0
5,AS015315,5,0
6,AS015739,5,0
7,AS015746,2,0
8,AS019120,3,0
9,AS019860,5,0


In [21]:
use_log_weekly['routine_flag'] = use_log_weekly['routine_flag'].where(use_log_weekly['count']<4, 1)
user_type = pd.merge(user_type, use_log_weekly, how='left', on='customer_id')
user_type.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,count,routine_flag
0,OA832399,XXXX,C01,F,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반,4.833333,5.0,8,2,4,1
1,PL270116,XXXXX,C01,M,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반,5.083333,5.0,7,3,5,1
2,OA974876,XXXXX,C01,M,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반,4.583333,5.0,6,3,5,1
3,HD024127,XXXXX,C01,F,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반,4.833333,4.5,7,2,5,1
4,HD661448,XXXXX,C03,F,2015-05-01,NaT,CA1,0,2_야간,6000,2_일반,3.916667,4.0,6,1,5,1


### 테크닉27 : 고객 데이터와 이용이력데이터를 결합하자

In [22]:
customer_join = pd.merge(use_log, user_type, how='left', on='customer_id')
customer_join.head(20)

,log_id,customer_id,usedate,연월,weekday,name,class,gender,start_date,end_date,...,is_deleted,class_name,price,campaign_name,mean,median,max,min,count,routine_flag
0,L00000049012330,AS009373,2018-04-01,201804,6,XX,C01,F,2015-11-01,NaT,...,0,0_종일,10500,2_일반,5.083333,5.0,7,3,5,1
1,L00000049012331,AS015315,2018-04-01,201804,6,XXXXX,C01,M,2015-07-01,NaT,...,0,0_종일,10500,2_일반,4.833333,5.0,7,3,5,1
2,L00000049012332,AS040841,2018-04-01,201804,6,XXXXX,C01,F,2017-05-01,NaT,...,0,0_종일,10500,2_일반,6.000000,6.0,10,4,5,1
3,L00000049012333,AS046594,2018-04-01,201804,6,XXX,C01,M,2018-03-01,NaT,...,0,0_종일,10500,2_일반,7.166667,7.0,10,5,5,1
4,L00000049012334,AS073285,2018-04-01,201804,6,XXXX,C01,M,2016-09-01,NaT,...,0,0_종일,10500,2_일반,4.916667,4.5,8,2,5,1
5,L00000049012335,AS077151,2018-04-01,201804,6,XXX,C02,F,2016-01-01,NaT,...,0,1_주간,7500,2_일반,5.166667,5.0,9,3,5,1
6,L00000049012336,AS161071,2018-04-01,201804,6,XXXXX,C01,M,2018-02-01,NaT,...,0,0_종일,10500,2_일반,6.833333,8.0,9,2,5,1
7,L00000049012337,AS170379,2018-04-01,201804,6,XXXX,C01,M,2017-03-01,NaT,...,0,0_종일,10500,2_일반,6.083333,6.0,9,1,5,1
8,L00000049012338,AS186727,2018-04-01,201804,6,XXXXX,C03,M,2017-03-01,NaT,...,0,2_야간,6000,2_일반,5.083333,5.0,8,3,5,1
9,L00000049012339,AS193568,2018-04-01,201804,6,XXXXX,C01,F,2018-02-01,NaT,...,0,0_종일,10500,2_일반,6.583333,7.0,10,3,5,1


### 테크닉28 : 회원기간을 계산하자

In [30]:
from dateutil.relativedelta import relativedelta

In [31]:
customer_join['calc_date'] = customer_join['end_date']
customer_join

,log_id,customer_id,usedate,연월,weekday,name,class,gender,start_date,end_date,...,class_name,price,campaign_name,mean,median,max,min,count,routine_flag,calc_date
0,L00000049012330,AS009373,2018-04-01,201804,6,XX,C01,F,2015-11-01,NaT,...,0_종일,10500,2_일반,5.083333,5.0,7,3,5,1,NaT
1,L00000049012331,AS015315,2018-04-01,201804,6,XXXXX,C01,M,2015-07-01,NaT,...,0_종일,10500,2_일반,4.833333,5.0,7,3,5,1,NaT
2,L00000049012332,AS040841,2018-04-01,201804,6,XXXXX,C01,F,2017-05-01,NaT,...,0_종일,10500,2_일반,6.000000,6.0,10,4,5,1,NaT
3,L00000049012333,AS046594,2018-04-01,201804,6,XXX,C01,M,2018-03-01,NaT,...,0_종일,10500,2_일반,7.166667,7.0,10,5,5,1,NaT
4,L00000049012334,AS073285,2018-04-01,201804,6,XXXX,C01,M,2016-09-01,NaT,...,0_종일,10500,2_일반,4.916667,4.5,8,2,5,1,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197423,L00000049209753,TS977703,2019-03-31,201903,6,XX,C01,M,2018-04-15,NaT,...,0_종일,10500,1_입회비무료,7.000000,7.5,8,4,5,1,NaT
197424,L00000049209754,TS979550,2019-03-31,201903,6,XXXXXX,C03,M,2018-10-08,NaT,...,2_야간,6000,2_일반,8.000000,7.5,10,7,5,1,NaT
197425,L00000049209755,TS995299,2019-03-31,201903,6,XXXXXX,C03,M,2015-09-01,NaT,...,2_야간,6000,2_일반,4.500000,4.5,7,2,5,1,NaT
197426,L00000049209756,TS995853,2019-03-31,201903,6,XXXX,C01,M,2019-02-08,NaT,...,0_종일,10500,2_일반,9.500000,9.5,11,8,5,1,NaT


In [34]:
customer_join['calc_date'] = customer_join['calc_date'].fillna(pd.to_datetime('20190430'))
customer_join['membership_period'] = 0

customer_join

,log_id,customer_id,usedate,연월,weekday,name,class,gender,start_date,end_date,...,price,campaign_name,mean,median,max,min,count,routine_flag,calc_date,membership_period
0,L00000049012330,AS009373,2018-04-01,201804,6,XX,C01,F,2015-11-01,NaT,...,10500,2_일반,5.083333,5.0,7,3,5,1,2019-04-30,0
1,L00000049012331,AS015315,2018-04-01,201804,6,XXXXX,C01,M,2015-07-01,NaT,...,10500,2_일반,4.833333,5.0,7,3,5,1,2019-04-30,0
2,L00000049012332,AS040841,2018-04-01,201804,6,XXXXX,C01,F,2017-05-01,NaT,...,10500,2_일반,6.000000,6.0,10,4,5,1,2019-04-30,0
3,L00000049012333,AS046594,2018-04-01,201804,6,XXX,C01,M,2018-03-01,NaT,...,10500,2_일반,7.166667,7.0,10,5,5,1,2019-04-30,0
4,L00000049012334,AS073285,2018-04-01,201804,6,XXXX,C01,M,2016-09-01,NaT,...,10500,2_일반,4.916667,4.5,8,2,5,1,2019-04-30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197423,L00000049209753,TS977703,2019-03-31,201903,6,XX,C01,M,2018-04-15,NaT,...,10500,1_입회비무료,7.000000,7.5,8,4,5,1,2019-04-30,0
197424,L00000049209754,TS979550,2019-03-31,201903,6,XXXXXX,C03,M,2018-10-08,NaT,...,6000,2_일반,8.000000,7.5,10,7,5,1,2019-04-30,0
197425,L00000049209755,TS995299,2019-03-31,201903,6,XXXXXX,C03,M,2015-09-01,NaT,...,6000,2_일반,4.500000,4.5,7,2,5,1,2019-04-30,0
197426,L00000049209756,TS995853,2019-03-31,201903,6,XXXX,C01,M,2019-02-08,NaT,...,10500,2_일반,9.500000,9.5,11,8,5,1,2019-04-30,0


In [36]:
for i in range(len(customer_join)):
    delta =  relativedelta(customer_join['calc_date'].iloc[i],customer_join['start_date'].iloc[i])
    customer_join["membership_period"].iloc[i] = delta.years*12 + delta.months
customer_join.head()

,log_id,customer_id,usedate,연월,weekday,name,class,gender,start_date,end_date,...,price,campaign_name,mean,median,max,min,count,routine_flag,calc_date,membership_period
0,L00000049012330,AS009373,2018-04-01,201804,6,XX,C01,F,2015-11-01,NaT,...,10500,2_일반,5.083333,5.0,7,3,5,1,2019-04-30,41
1,L00000049012331,AS015315,2018-04-01,201804,6,XXXXX,C01,M,2015-07-01,NaT,...,10500,2_일반,4.833333,5.0,7,3,5,1,2019-04-30,45
2,L00000049012332,AS040841,2018-04-01,201804,6,XXXXX,C01,F,2017-05-01,NaT,...,10500,2_일반,6.000000,6.0,10,4,5,1,2019-04-30,23
3,L00000049012333,AS046594,2018-04-01,201804,6,XXX,C01,M,2018-03-01,NaT,...,10500,2_일반,7.166667,7.0,10,5,5,1,2019-04-30,13
4,L00000049012334,AS073285,2018-04-01,201804,6,XXXX,C01,M,2016-09-01,NaT,...,10500,2_일반,4.916667,4.5,8,2,5,1,2019-04-30,31


### 테크닉29 : 고객행동의 각종통계량을 파악하자

### 테크닉30 : 탈퇴회원과 지속회원의 차이를 파악하자